In [1]:
# imports
from z3 import *
from utils import *
import numpy as np

In [2]:
instance00 = {
    "m" : 1,
    "n" : 2,
    "l" : [15],
    "s" : [3, 2],
    "distances" : [
        [0, 4, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

instance0 = {
    "m" : 2,
    "n" : 2,
    "l" : [15, 10],
    "s" : [3, 2],
    "distances" : [
        [0, 3, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

instance1 = {
    "m" : 3,
    "n" : 7,
    "l" : [15, 10, 7],
    "s" : [3, 2, 6, 8, 5, 4, 4],
    "distances" : [
        [0, 3, 3, 6, 5, 6, 6, 2],
        [3, 0, 4, 3, 4, 7, 7, 3],
        [3, 4, 0, 7, 6, 3, 5, 3],
        [6, 3, 7, 0, 3, 6, 6, 4],
        [5, 4, 6, 3, 0, 3, 3, 3],
        [6, 7, 3, 6, 3, 0, 2, 4],
        [6, 7, 5, 6, 3, 2, 0, 4],
        [2, 3, 3, 4, 3, 4, 4, 0]
    ]
}

instance2 = {
    "m" : 2,
    "n" : 3,
    "l" : [15, 10],
    "s" : [10, 2, 4],
    "distances" : [
        [0, 3, 3, 1],
        [3, 0, 4, 1],
        [3, 4, 0, 1],
        [3, 2, 1, 0]
    ]
}

instance12 = {
    "m" : 5,
    "n" : 10,
    "l" : [10, 10, 19, 12, 11],
    "s" : [6, 3, 2, 6, 4, 5, 3, 6, 4, 3],
    "distances" : [
                [ 0, 5, 3, 2, 2, 2, 4, 2, 3, 1, 5 ], 
                [ 2, 0, 1, 1, 1, 2, 2, 2, 3, 4, 2 ], 
                [ 4, 2, 0, 4, 2, 2, 2, 5, 4, 1, 2 ], 
                [ 4, 2, 3, 0, 3, 3, 3, 1, 5, 4, 1 ], 
                [ 4, 2, 4, 4, 0, 1, 4, 5, 2, 5, 2 ], 
                [ 2, 4, 5, 1, 3, 0, 1, 3, 5, 4, 3 ], 
                [ 5, 4, 3, 3, 5, 4, 0, 3, 5, 1, 2 ], 
                [ 4, 3, 2, 2, 5, 5, 2, 0, 5, 5, 1 ], 
                [ 1, 5, 3, 4, 1, 1, 4, 1, 0, 2, 3 ], 
                [ 4, 2, 2, 2, 1, 3, 4, 3, 1, 0, 4 ], 
                [ 4, 5, 5, 4, 3, 5, 5, 1, 5, 5, 0 ] 
            ]
}

instance3 = {  
    "m" : 6,
    "n" : 12,
    "l" : [28,28,25,27,29,28],
    "s" : [7,13,5,8,6,10,4,9,3,5,3,13],
    "distances" : [
        [0,2,10,5,9,10,3,5,2,2,6,8,9],
        [2,0,6,7,6,10,4,9,8,8,9,5,1],
        [9,1,0,2,7,1,10,8,6,4,6,2,4],
        [10,4,4,0,3,9,8,2,2,6,9,8,2],
        [5,9,5,2,0,9,6,9,4,10,9,10,5],
        [8,2,10,7,6,0,10,4,5,3,4,3,1],
        [10,5,8,2,2,3,0,3,1,2,9,7,9],
        [5,9,4,4,10,7,10,0,5,8,8,6,2],
        [6,10,2,8,10,6,4,4,0,1,5,2,4],
        [6,3,6,7,1,2,3,4,1,0,10,9,10],
        [2,1,2,4,10,10,2,7,2,6,0,2,1],
        [10,1,4,3,2,8,4,1,1,9,7,0,10],
        [2,5,2,4,2,5,6,7,3,1,9,8,0]
    ]
}

In [3]:

def min_index(array):
    for i in range(len(array)):
        if array[i] == True:
            return i
    return 0

def max_index(array):
    for i in range(len(array)-1, -1, -1):
        if array[i] == True:
            return i
    return len(array)

def indeces(array):
    indeces = []
    indeces.append(len(array))
    for i in range(len(array)-1, -1, -1):
        if array[i] == True:
            indeces.append(i)
    return indeces


In [9]:
def MCP(instance):
    m = instance["m"] # couriers
    n = instance["n"] # packages
    l = instance["l"] # weigths
    s = instance["s"] # sizes of couriers
    capacities_check_array = instance["lowerequal_matrix"]

    solver = Solver()

    # Variables
    # To codify that courier i deliver package j at time k
    v = [[[Bool(f"x_{i}_{j}_{k}") for k in range(n+2)] for j in range (n+1)] for i in range(m)]

    d = [[[Bool(f"d_{i}_{start}_{end}") for end in range(n+1)]
          for start in range(n+1)] for i in range(m)]

    for i in range(m):
            for k in range(n):
                for startj in range(n+1):
                    for endj in range(n+1):
                        solver.add(
                            Implies(And(v[i][startj][k], v[i][endj][k+1]), d[i][startj][endj])
                        )

    # Constraints
    # 1. Each courier can carry at most l[i] kg
    for i in range (m):
        weigth_set = []
        for j in range (n):
            for k in range(1,n+1):
                for _ in range(s[j]):
                    weigth_set.append(v[i][j][k])
        solver.add(at_most_k_seq(weigth_set, l[i], f"courier_{i}_load"))

    # 2. Each courier i starts and end at position j = n
    for i in range (m):
        solver.add(And(v[i][n][0], v[i][n][n+1]))


    # 3. Each courier can't be in two places at the same time 
    for i in range(m):
        for k in range(n+2):
            solver.add(exactly_one_bw([v[i][j][k] for j in range(n+1)], f"amo_package_{i}_{k}"))
    
    # 4. Each package j is delivered exactly once
    for j in range(n):
        solver.add(exactly_one_bw([v[i][j][k] for k in range(1,n+1) for i in range(m)], f"exactly_once_{j}"))
    
    # Symmetry breaking constraints
    # 5. once a courier arrive to depot (j = n+1), it can't depart from there
    for i in range (m):
        for k in range(1, n):
            solver.add(
                Implies(v[i][n][k], v[i][n][k+1])
            )
    
    # symmetry breaking for courriers with equal capacity
    # 6. if the courrier i have lower capacity than currier i1, then what can be delivered by i cant be delivered by i1
    #if capacities_check_array[i][i1] is true, then v[i][:][k] should be lexicographically less than v[i1][:][k]
    assignments_array = [[Bool(f"aa_{i}_{j}") for j in range(n)] for i in range (m)]

    # aa true if the package j is assigned to i at any k
    for i in range(m):
        for j in range(n):
            solver.add(
                assignments_array[i][j] == Or([v[i][j][k] for k in range(1,n+1)]) 
            )

    for i in range(m):
        for i1 in range(m):
            for j in range(1,min_index([assignments_array[i][jj] for jj in range(n)])+1):
                solver.add(
                    Implies(
                        capacities_check_array[i][i1],
                        Implies( Or(And(assignments_array[i][j-1], assignments_array[i1][j-1]), And(Not(assignments_array[i][j-1]),Not(assignments_array[i1][j-1]))) ,
                            Or(And(assignments_array[i][j], assignments_array[i1][j]), And(Not(assignments_array[i][j]),Not(assignments_array[i1][j])), assignments_array[i][j])
                        )
                    )
                )

    return solver, v, d

def add_distance_constraint(solver, instance, d, upperBound):
    m = instance["m"] # couriers
    n = instance["n"] # packages
    distances = instance["distances"] # distances between packages
    # distance calculation
    # for i in range (m):
    #     dist_set = []

    #     for j1 in range (n+1):
    #         for j2 in range (n+1):
    #             for k in range(1,n+2):
    #                 and_constraint = And(v[i][j1][k-1], v[i][j2][k])
    #                 for _ in range(distances[j1][j2]):
    #                     dist_set.append(and_constraint)

    #     solver.add(at_most_k_seq(dist_set, upperBound, f"courier_{i}_distmax_{upperBound}"))
    for i in range(m):
        dists = []
        for start in range(n+1):
            for end in range(n+1):
                for z in range(distances[start][end]):
                    dists.append(d[i][start][end])
        solver.add(at_most_k_seq(dists, upperBound, f"Courier_dist_{i}_{upperBound}"))
    
def solve_mcp(solver, instance, v, verbose=False):
    m = instance["m"] # couriers
    n = instance["n"] # packages
    if not verbose:
        if solver.check() == sat:
            return solver.model()
        else:  
            return False
    else:
        print(solver.check())
        model = solver.model()
        if solver.check() == sat:
            for i in range(m):
                print()
                print(f"Courier {i+1}:")
                for k in range(n+2):
                    for j in range(n+1):
                        if model[v[i][j][k]]:
                            print(f"Time {k} Place {j} : {model[v[i][j][k]]}")


In [5]:
# Second version using Pseudo-Boolean constraints
def MCP_Pb(instance):
    m = instance["m"] # courriers
    n = instance["n"] # packages
    l = instance["l"] # weigths
    s = instance["s"] # sizes of courriers
    capacities_check_array = instance["lowerequal_matrix"]

    solver = Solver()

    # Variables
    # To codify that courrier i deliver package j at time k
    v = [[[Bool(f"x_{i}_{j}_{k}") for k in range(n+2)] for j in range (n+1)] for i in range(m)]

    d = [[[Bool(f"d_{i}_{start}_{end}") for end in range(n+1)]
          for start in range(n+1)] for i in range(m)]

    for i in range(m):
            for k in range(n):
                for startj in range(n+1):
                    for endj in range(n+1):
                        solver.add(
                            Implies(And(v[i][startj][k], v[i][endj][k+1]), d[i][startj][endj])
                        )

    # Constraints
    # 1. Each courier can carry at most l[i] kg
    # Pb version
    for i in range(m):
       solver.append(PbLe([(v[i][j][k],s[j]) for j in range(n) for k in range(1,n+1)], l[i]))


    # 2. Each courier i starts and ends at position j = n
    for i in range(m):
        solver.add(And(v[i][n][0], v[i][n][n+1]))


    # 3. Each courier can't be in two places at the same time 
    for i in range(m):
        for k in range(n+2):
            solver.add(exactly_one_seq([v[i][j][k] for j in range(n+1)], f"amo_package_{k}_{i}")) #(PbEq([(v[i][j][k],1) for j in range(n+1)],1))
    
    # 4. Each package j is delivered exactly once
    for j in range(n):
        solver.add(exactly_one_seq([v[i][j][k] for k in range(1,n+1) for i in range(m)], f"exactly_once_{j}")) #(PbEq([(v[i][j][k],1) for k in range(1,n+1) for i in range(m)],1))    
    
    # Symmetry breaking constraints
    # 5. once a courier arrive to depot (j = n+1), it can't depart from there
    for i in range (m):
        for k in range(1, n):
            solver.add(
                Implies(v[i][n][k], v[i][n][k+1])
            )
    
    # symmetry breaking for courriers with equal capacity
    assignments_array = [[Bool(f"aa_{i}_{j}") for j in range(n)] for i in range (m)]

    # aa true if the package j is assigned to i at any k
    for i in range(m):
        for j in range(n):
            solver.add(
                assignments_array[i][j] == Or([v[i][j][k] for k in range(1,n+1)]) 
            )

    for i in range(m):
        for i1 in range(m):
            for j in range(1,min_index([assignments_array[i][jj] for jj in range(n)])+1):
                solver.add(
                    Implies(
                        capacities_check_array[i][i1],
                        Implies( Or(And(assignments_array[i][j-1], assignments_array[i1][j-1]), And(Not(assignments_array[i][j-1]),Not(assignments_array[i1][j-1]))) ,
                            Or(And(assignments_array[i][j], assignments_array[i1][j]), And(Not(assignments_array[i][j]),Not(assignments_array[i1][j])), assignments_array[i][j])
                        )
                    )
                )

    return solver, v, d

def add_distance_constraint_pb(solver, instance, d, upperBound):
    m = instance["m"] # courriers
    n = instance["n"] # packages
    distances = instance["distances"] # distances between packages

    # distance calculation
    # Pb version
    for i in range(m):
        #solver.append(PbLe([(And(v[i][j1][k-1],v[i][j2][k]),distances[j1][j2]) for j1 in range (n+1) for j2 in range (n+1) for k in range(1,n+2)], upperBound))
        solver.append(PbLe([(d[i][j1][j2],distances[j1][j2]) for j1 in range (n+1) for j2 in range (n+1)], upperBound))


In [11]:
def MultipleCouriersPlanning(instance, usePb = False):

    if usePb:
        mcp = MCP_Pb
        add_constraint = add_distance_constraint_pb
    else:
        mcp = MCP
        add_constraint = add_distance_constraint

    m = instance["m"]
    l = instance["l"]
    lowerequal_matrix = np.full((m,m), Bool(False))
    for i in range(m):
        for i1 in range(m):
            if l[i] <= l[i1]:
                lowerequal_matrix[i,i1] = True
    instance["lowerequal_matrix"] = lowerequal_matrix

    n = instance["n"]
    distances = np.array(instance["distances"])

    upperBound1 = distances[n, 0] + np.sum(distances[np.arange(n), np.arange(n) + 1])
    upperBound2 = distances[0, n] + np.sum(distances[np.arange(n) + 1, np.arange(n)])
    upperBound = min(upperBound1, upperBound2)

    originalUpperBound = upperBound

    fromDepot = distances[n, :n]
    toDepot = distances[:n, n]
    inMiddle = distances[:n, :n]
    inMiddle = inMiddle[~np.eye(inMiddle.shape[0], dtype=bool)]
    numMiddle = int(np.ceil(instance["m"] / n)) - 1

    lowerBound = np.min(fromDepot) + np.min(toDepot) + numMiddle * np.min(inMiddle)
    pivot = (upperBound + lowerBound) // 2

    solver, v, d = mcp(instance) # create basic solver

    # binary search using bounds
    print("searching...")
    while True:
        if lowerBound == upperBound:
            return upperBound
        # print("lower:",lowerBound)
        # print("upper:",upperBound)
        print("pivot:",pivot)

        solver.push()
        add_constraint(solver, instance, d, pivot)

        res = solve_mcp(solver, instance, v, verbose=False)
        if(res == False):
            print("fail")
            if lowerBound > upperBound:
                if upperBound == originalUpperBound:
                    return unsat
                else:
                    return upperBound
            lowerBound = pivot + 1

            solver.pop()

            pivot = (upperBound + lowerBound) // 2
        else:
            print("success")
            upperBound = pivot
            pivot = (upperBound + lowerBound) // 2

In [12]:
usePb = False
verbose = False
instance = instance1
result = MultipleCouriersPlanning(instance, usePb)
if result == unsat:
    print("\nResult of the computation: unsat")
else:    
    print("\nResult of the minimization: ",result)
    if verbose:
        if usePb:
            MCP_Pb(instance,result, verbose)
        else:
            MCP(instance,result, verbose)
    

searching...
pivot: 16
success
pivot: 10
fail
pivot: 13
success
pivot: 12
success
pivot: 11
fail

Result of the minimization:  12


In [7]:
usePb = True
verbose = False
instance = instance1
result = MultipleCouriersPlanning(instance, usePb)
if result == unsat:
    print("\nResult of the computation: unsat")
else:    
    print("\nResult of the minimization: ",result)
    if verbose:
        if usePb:
            MCP_Pb(instance,result, verbose)
        else:
            MCP(instance,result, verbose)
    

searching...
pivot: 16
success
pivot: 10
fail
pivot: 13
success
pivot: 12
success
pivot: 11
fail

Result of the minimization:  12


In [17]:
instance = instance00

m = instance["m"]
l = instance["l"]
lowerequal_matrix = np.full((m,m), Bool(False))
for i in range(m):
    for i1 in range(m):
        if l[i] <= l[i1]:
            lowerequal_matrix[i,i1] = True
instance["lowerequal_matrix"] = lowerequal_matrix

MCP(instance);